# Spot Robot Training on Google Colab (Isaac Sim)

**GPU:** T4 (free tier)  
**What this does:** Trains Spot with 5 depth cameras to navigate indoor environments.

## Cell Order:
1. Install dependencies
2. Clone repo
3. Mount Google Drive
4. Verify GPU
5. **Set your parameters** (override defaults or skip to use defaults)
6. Train
7. Resume training (only if disconnected)
8. Evaluate & record video
9. Play video
10. List saved files

## Cell 1: Install Dependencies

In [ ]:
!pip install gymnasium stable-baselines3[extra] torch tensorboard imageio[ffmpeg] numpy -q
print("All packages installed!")

## Cell 2: Clone Your Repo

In [ ]:
import os

if not os.path.exists('/content/claude_code'):
    !git clone https://github.com/Raj9408612613/claude_code.git /content/claude_code
    print("Repo cloned!")
else:
    !cd /content/claude_code && git pull
    print("Repo already exists, pulled latest.")

# Verify spot_isaac_sim exists
!ls /content/claude_code/spot_isaac_sim/

## Cell 3: Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

SAVE_DIR = '/content/drive/MyDrive/robot_training/models'
LOG_DIR  = '/content/drive/MyDrive/robot_training/logs'
VIDEO_DIR = '/content/drive/MyDrive/robot_training/videos'

os.makedirs(SAVE_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)
os.makedirs(VIDEO_DIR, exist_ok=True)

print(f"Models  -> {SAVE_DIR}")
print(f"Logs    -> {LOG_DIR}")
print(f"Videos  -> {VIDEO_DIR}")

## Cell 4: Verify GPU

In [ ]:
import torch

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_mem = torch.cuda.get_device_properties(0).total_mem / 1e9
    print(f"GPU: {gpu_name} ({gpu_mem:.1f} GB)")
else:
    print("WARNING: No GPU! Go to Runtime > Change runtime type > T4 GPU")

## Cell 5: Set Your Parameters

**This is the key cell.** Pass only the values you want to change.  
Anything you don't pass keeps its default from `config.py`.

| Parameter | Default | Small Budget |
|-----------|---------|-------------|
| `total_timesteps` | 10,000,000 | 500,000 |
| `max_episode_steps` | 1,000 | 500 |
| `combined_mlp_layers` | [256, 128] | [128, 128] |
| `checkpoint_freq` | 100,000 | 50,000 |
| `n_envs` | 4 | 4 |
| `goal_distance_range` | [2, 10] | [2, 5] |

In [ ]:
import sys
sys.path.insert(0, '/content/claude_code')

from spot_isaac_sim import config

# =============================================
# CHANGE ONLY WHAT YOU NEED — DELETE THE REST
# Anything not listed here keeps its default.
# =============================================
config.apply_overrides(
    # Training scale
    total_timesteps=500_000,         # default: 10,000,000
    max_episode_steps=500,           # default: 1,000
    checkpoint_freq=50_000,          # default: 100,000

    # Network (smaller = faster on T4)
    combined_mlp_layers=[128, 128],  # default: [256, 128]

    # Environment (easier = learns faster)
    goal_distance_range=[2.0, 5.0],  # default: [2.0, 10.0]

    # Save to Google Drive
    save_dir=SAVE_DIR,
    log_dir=LOG_DIR,
)

# Print what's active so you can verify
config.print_active_config()

## Cell 6: TRAIN

~30-60 min on T4 for 500k steps. Checkpoints save to Google Drive automatically.

In [ ]:
from spot_isaac_sim.train import train

# All parameters come from config (set in Cell 5)
model = train(headless=True)

print("\nTraining complete!")
print(f"Models saved to: {SAVE_DIR}")

## Cell 7: Resume Training (only if Colab disconnected)

Skip this cell if training finished normally.

In [ ]:
import glob

checkpoints = sorted(glob.glob(f"{SAVE_DIR}/spot_nav_*.zip"))
if checkpoints:
    latest = checkpoints[-1]
    print(f"Resuming from: {latest}")

    from spot_isaac_sim.train import train
    model = train(headless=True, continue_from=latest)
else:
    print("No checkpoints found. Run Cell 6 first.")

## Cell 8: Evaluate & Record Video

In [ ]:
import numpy as np
import imageio
from stable_baselines3 import PPO
from spot_isaac_sim.navigation_env import SpotNavigationEnv

# Load best model
best_path = f"{SAVE_DIR}/best_model.zip"
if not os.path.exists(best_path):
    best_path = f"{SAVE_DIR}/spot_nav_final.zip"

print(f"Loading: {best_path}")
model = PPO.load(best_path)

env = SpotNavigationEnv(headless=True)

# Run 3 episodes
for episode in range(3):
    obs, info = env.reset()
    episode_reward = 0
    steps = 0

    for step in range(500):
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, terminated, truncated, info = env.step(action)
        episode_reward += reward
        steps += 1
        if terminated or truncated:
            break

    dist = info.get('distance_to_goal', '?')
    goal = info.get('goal_reached', False)
    print(f"Episode {episode+1}: reward={episode_reward:.1f}, steps={steps}, "
          f"dist={dist:.2f}m, goal_reached={goal}")

env.close()
print("\nEvaluation complete!")

## Cell 9: Training Stats

In [ ]:
import glob

print("Saved models:")
for f in sorted(glob.glob(f"{SAVE_DIR}/*.zip")):
    size_mb = os.path.getsize(f) / 1e6
    print(f"  {os.path.basename(f)} ({size_mb:.1f} MB)")

print(f"\nAll files on Google Drive — safe from disconnects!")